# Bench Mark

In [ ]:
import pandas as pd
import numpy as np

items_df = pd.read_csv('Data/items.csv')
shops_df = pd.read_csv('Data/shops.csv')
icats_df = pd.read_csv("Data/item_categories.csv")
train_df = pd.read_csv("Data/sales_train.csv.gz")
test_df  = pd.read_csv('Data/test.csv.gz') # 214200 rows

In [ ]:
test_shops = test_df.shop_id.unique()
train_df = train_df[train_df.shop_id.isin(test_shops)]
test_items = test_df.item_id.unique()
train_df = train_df[train_df.item_id.isin(test_items)]

print('train:', train_df.shape, 'test:', test_df.shape, 'items:', items_df.shape, 'shops:', shops_df.shape)

In [ ]:
test_only = test_df[~test_df['item_id'].isin(train_df['item_id'].unique())]['item_id'].unique()

In [ ]:
# group by
train_grp = train_df.groupby(['date_block_num','shop_id','item_id'])

In [ ]:
# price mean by month
train_price = pd.DataFrame(train_grp.mean()['item_price']).reset_index()
train_price.head()

In [ ]:
# count summary by month
train_monthly = pd.DataFrame(train_grp.sum()['item_cnt_day']).reset_index()
train_monthly.rename(columns={'item_cnt_day':'item_cnt'}, inplace=True)
train_monthly.head()

In [ ]:
train_piv = train_df.pivot_table(index=['shop_id','item_id'], columns='date_block_num', values='item_cnt_day',aggfunc='sum').fillna(0.0)    
train_piv = train_piv.reset_index()
train_piv.head()

In [ ]:
grp = train_monthly.groupby(['shop_id', 'item_id'])
train_shop = grp.agg({'item_cnt':['mean','median','std']}).reset_index()
train_shop.columns = ['shop_id','item_id','cnt_mean_shop','cnt_med_shop','cnt_std_shop']
train_shop.head()

In [ ]:
train_cat_monthly = pd.merge(train_monthly, items_df, on=['item_id'], how='left')
grp = train_cat_monthly.groupby(['shop_id', 'item_category_id'])
train_shop_cat = grp.agg({'item_cnt':['mean']}).reset_index()
train_shop_cat.columns = ['shop_id','item_category_id','cnt_mean_cat_shop']
train_shop_cat.head()

In [ ]:
train_last = train_monthly[train_monthly['date_block_num']==33]
train_last = train_last.drop(['date_block_num'], axis=1).rename(columns={'item_cnt':'cnt_sum_last'})
train_last.head()

In [ ]:
# Prev month
train_prev = train_monthly.copy()
train_prev['date_block_num'] = train_prev['date_block_num'] + 1
train_prev = train_prev.rename(columns={'item_cnt':'cnt_sum_prev'})
train_prev.head()

In [ ]:
train_cat_prev = pd.merge(train_prev, items_df, on=['item_id'], how='left')
grp = train_cat_prev.groupby(['date_block_num','shop_id','item_category_id'])
train_cat_prev = grp['cnt_sum_prev'].sum().reset_index()
train_cat_prev = train_cat_prev.rename(columns={'cnt_sum_prev':'cnt_sum_cat_prev'})
train_cat_prev.head()

In [ ]:
col = np.arange(34)
pivT = train_piv[col].T
evm_s = pivT.ewm(span=12).mean().T
evm_l = pivT.ewm(span=26).mean().T

macd = evm_s - evm_l
sig = macd.ewm(span=9).mean()

train_piv_key = train_piv.loc[:,['shop_id','item_id']]
train_evm_list = []

for c in col:
    sub_evm_s = pd.DataFrame(evm_s.loc[:,c]).rename(columns={c:'cnt_evm_s_prev'})
    sub_evm_l = pd.DataFrame(evm_l.loc[:,c]).rename(columns={c:'cnt_evm_l_prev'})
    sub_macd = pd.DataFrame(macd.loc[:,c]).rename(columns={c:'cnt_macd_prev'})
    sub_sig = pd.DataFrame(sig.loc[:,c]).rename(columns={c:'cnt_sig_prev'})
    
    sub_evm = pd.concat([train_piv_key, sub_evm_s, sub_evm_l, sub_macd, sub_sig], axis=1)
    sub_evm['date_block_num'] = c + 1
    train_evm_list.append(sub_evm)
    
train_evm_prev = pd.concat(train_evm_list)
#train_evm_prev.head()
train_evm_prev.query("shop_id == 2 & item_id == 30").tail()

In [ ]:
icats_df['item_category_group'] = icats_df['item_category_name'].apply(lambda x: str(x).split(' ')[0])
icats_df['item_category_group'] = pd.Categorical(icats_df['item_category_group']).codes

item_cats = pd.merge(icats_df, pd.get_dummies(icats_df['item_category_group'], prefix='item_category_group', drop_first=True), left_index=True, right_index=True)
item_cats.drop(['item_category_group'], axis=1, inplace=True)

shops_df['city'] = shops_df.shop_name.apply(lambda x: str.replace(x, '!', '')).apply(lambda x: x.split(' ')[0])
shops_df['city'] = pd.Categorical(shops_df['city']).codes

In [ ]:
def mergeFeature(source): 
    d = source
    d = pd.merge(d, items_df, on=['item_id'], how='left')
    d = pd.merge(d, item_cats, on=['item_category_id'], how='left')
    d = pd.merge(d, shops_df, on=['shop_id'], how='left')

    d = pd.merge(d, train_price, on=['date_block_num','shop_id','item_id'], how='left')
    d = pd.merge(d, train_shop, on=['shop_id','item_id'], how='left')
    #d = pd.merge(d, train_shop_cat, on=['shop_id','item_category_id'], how='left')
    #d = pd.merge(d, train_last, on=['shop_id','item_id'], how='left')
    d = pd.merge(d, train_prev, on=['date_block_num','shop_id','item_id'], how='left')
    d = pd.merge(d, train_evm_prev, on=['date_block_num','shop_id','item_id'], how='left')
    d = pd.merge(d, train_cat_prev, on=['date_block_num','shop_id','item_category_id'], how='left')

    d.drop(['shop_id','shop_name','item_id','item_name','item_category_id','item_category_name'], axis=1, inplace=True)
    d.fillna(0.0, inplace=True)
    return d


In [ ]:
train_set = mergeFeature(train_monthly)

In [ ]:
test_df['date_block_num'] = 34

X_test = mergeFeature(test_df.drop(['ID'], axis=1))

In [ ]:
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_squared_error

X_train = train_set.drop(['item_cnt'], axis=1)
Y_train = train_set['item_cnt']

xgb = XGBRegressor(n_estimators=25, max_depth=12, learning_rate=0.1, subsample=1, colsample_bytree=1, eval_metric='rmse')

xgb.fit(X_train, Y_train)

preds = xgb.predict(X_train)

print r2_score(Y_train, preds) #0.955330774186
print mean_squared_error(Y_train, preds) #5.69332522603

# Modification

In [1]:
import pandas as pd
import numpy as np

items_df = pd.read_csv('Data/items.csv')
shops_df = pd.read_csv('Data/shops.csv')
icats_df = pd.read_csv("Data/item_categories.csv")
train_df = pd.read_csv("Data/sales_train.csv.gz")
test_df  = pd.read_csv('Data/test.csv.gz') 

test_shops = test_df.shop_id.unique()
train_df = train_df[train_df.shop_id.isin(test_shops)]
test_items = test_df.item_id.unique()
train_df = train_df[train_df.item_id.isin(test_items)]

print('train:', train_df.shape, 'test:', test_df.shape, 'items:', items_df.shape, 'shops:', shops_df.shape)

('train:', (1224439, 6), 'test:', (214200, 3), 'items:', (22170, 3), 'shops:', (60, 2))


In [2]:
train_grp = train_df.groupby(['date_block_num','shop_id','item_id'])

train_price = pd.DataFrame(train_grp.mean()['item_price']).reset_index()

test_grp = train_df.groupby(['shop_id', 'item_id'])

test_price = pd.DataFrame(test_grp.mean()['item_price']).reset_index()

test_df['date_block_num'] = 34
test_df['season'] = np.sin(34 * np.pi * 2/12.0)

a = test_df[['date_block_num', 'shop_id', 'item_id']]
a = pd.merge(a, test_price, how='left', on=['shop_id', 'item_id'])
train_price = pd.concat((train_price, a), ignore_index=True)

train_monthly = pd.DataFrame(train_grp.sum()['item_cnt_day']).reset_index()

train_monthly.rename(columns={'item_cnt_day':'item_cnt'}, inplace=True)

train_monthly['season'] = np.sin(train_monthly['date_block_num']*2*np.pi/12.0)

In [3]:
train_piv = train_df.pivot_table(index=['shop_id','item_id'], columns='date_block_num', values='item_cnt_day',aggfunc='sum').fillna(0.0)    
train_piv = train_piv.reset_index()

grp = train_monthly.groupby(['shop_id', 'item_id'])
train_shop = grp.agg({'item_cnt':['mean','median','std']}).reset_index()
train_shop.columns = ['shop_id','item_id','cnt_mean_shop','cnt_med_shop','cnt_std_shop']

In [4]:
train_cat_monthly = pd.merge(train_monthly, items_df, on=['item_id'], how='left')
grp = train_cat_monthly.groupby(['shop_id', 'item_category_id'])
train_shop_cat = grp.agg({'item_cnt':['mean']}).reset_index()
train_shop_cat.columns = ['shop_id','item_category_id','cnt_mean_cat_shop']
train_shop_cat.head()

,shop_id,item_category_id,cnt_mean_cat_shop
0,2,2,1.807692
1,2,3,2.547170
2,2,5,1.600000
3,2,6,2.047244
4,2,7,1.333333


In [5]:
shops_df['city_id'] = shops_df.shop_name.apply(lambda x: str.replace(x, '!', '')).apply(lambda x: x.split(' ')[0])
shops_df['city_id'] = pd.Categorical(shops_df['city_id']).codes
shops_df.head(5)

,shop_name,shop_id,city_id
0,"!Якутск Орджоникидзе, 56 фран",0,29
1,"!Якутск ТЦ ""Центральный"" фран",1,29
2,"Адыгея ТЦ ""Мега""",2,0
3,"Балашиха ТРК ""Октябрь-Киномир""",3,1
4,"Волжский ТЦ ""Волга Молл""",4,2


In [6]:
# construct cnt_mean_cat_shop

train_cat_shop_monthly = pd.merge(train_cat_monthly, shops_df, on=['shop_id'], how='left')

In [7]:
grp = train_cat_shop_monthly.groupby(['city_id', 'item_category_id'])
train_city_cat = grp.agg({'item_cnt':['mean']}).reset_index()
train_city_cat.columns = ['city_id', 'item_category_id', 'cnt_mean_cat_city']
train_city_cat.head()

,city_id,item_category_id,cnt_mean_cat_city
0,0,2,1.807692
1,0,3,2.547170
2,0,5,1.600000
3,0,6,2.047244
4,0,7,1.333333


In [8]:
train_last = train_monthly[train_monthly['date_block_num']==33]
train_last = train_last.drop(['date_block_num'], axis=1).rename(columns={'item_cnt':'cnt_sum_last'})
train_last.head()

,shop_id,item_id,cnt_sum_last,season
571479,2,31,1.0,-1.0
571480,2,486,3.0,-1.0
571481,2,787,1.0,-1.0
571482,2,794,1.0,-1.0
571483,2,968,1.0,-1.0


In [9]:
# Prev month
train_prev = train_monthly.copy().drop(['season'], axis=1)
train_prev['date_block_num'] = train_prev['date_block_num'] + 1
train_prev = train_prev.rename(columns={'item_cnt':'cnt_sum_prev'})

train_cat_prev = pd.merge(train_prev, items_df, on=['item_id'], how='left')
grp = train_cat_prev.groupby(['date_block_num','shop_id','item_category_id'])
train_cat_prev = grp['cnt_sum_prev'].sum().reset_index()
train_cat_prev = train_cat_prev.rename(columns={'cnt_sum_prev':'cnt_sum_cat_prev'})
train_cat_prev.head()

,date_block_num,shop_id,item_category_id,cnt_sum_cat_prev
0,1,2,2,9.0
1,1,2,5,2.0
2,1,2,6,10.0
3,1,2,19,19.0
4,1,2,21,10.0


In [10]:
col = np.arange(34)
pivT = train_piv[col].T
evm_s = pivT.ewm(span=12).mean().T
evm_l = pivT.ewm(span=26).mean().T

macd = evm_s - evm_l
sig = macd.ewm(span=9).mean()

train_piv_key = train_piv.loc[:,['shop_id','item_id']]
train_evm_list = []

for c in col:
    sub_evm_s = pd.DataFrame(evm_s.loc[:,c]).rename(columns={c:'cnt_evm_s_prev'})
    sub_evm_l = pd.DataFrame(evm_l.loc[:,c]).rename(columns={c:'cnt_evm_l_prev'})
    sub_macd = pd.DataFrame(macd.loc[:,c]).rename(columns={c:'cnt_macd_prev'})
    sub_sig = pd.DataFrame(sig.loc[:,c]).rename(columns={c:'cnt_sig_prev'})
    
    sub_evm = pd.concat([train_piv_key, sub_evm_s, sub_evm_l, sub_macd, sub_sig], axis=1)
    sub_evm['date_block_num'] = c + 1
    train_evm_list.append(sub_evm)
    
train_evm_prev = pd.concat(train_evm_list)
#train_evm_prev.head()
train_evm_prev.query("shop_id == 2 & item_id == 30").tail()

,shop_id,item_id,cnt_evm_s_prev,cnt_evm_l_prev,cnt_macd_prev,cnt_sig_prev,date_block_num
0,2,30,0.037104,0.081511,-0.044407,-0.044407,30
0,2,30,0.031363,0.074861,-0.043498,-0.043498,31
0,2,30,0.026515,0.068800,-0.042284,-0.042284,32
0,2,30,0.022419,0.063267,-0.040848,-0.040848,33
0,2,30,0.018958,0.058211,-0.039253,-0.039253,34


In [11]:
train_cat_monthly.head(5)

,date_block_num,shop_id,item_id,item_cnt,season,item_name,item_category_id
0,0,2,33,1.0,0.0,1+1 (BD),37
1,0,2,482,1.0,0.0,1С:Бухгалтерия 8. Базовая версия,73
2,0,2,491,1.0,0.0,1С:Деньги 8,73
3,0,2,839,1.0,0.0,1С:Упрощенка 8,73
4,0,2,1007,3.0,0.0,3D Crystal Puzzle Замок XL,67


In [12]:
icats_df['item_category_group'] = icats_df['item_category_name'].apply(lambda x: str(x).split(' ')[0])
icats_df['item_category_group'] = pd.Categorical(icats_df['item_category_group']).codes

train_cat_group_monthly = pd.merge(train_cat_monthly, icats_df, on=['item_category_id'], how='left')

grp = train_cat_group_monthly.groupby(['item_category_group'])
train_group = grp.agg({'item_cnt':['mean', 'median', 'std']}).reset_index()
train_group.columns = ['item_category_group', 'cnt_mean_group', 'cnt_median_group', 'cnt_std_group']
train_group.head()

,item_category_group,cnt_mean_group,cnt_median_group,cnt_std_group
0,1,2.578591,1.0,3.366503
1,3,98.612022,45.0,199.144371
2,4,5.237907,2.0,12.195184
3,5,3.414953,2.0,8.123475
4,6,6.801076,4.0,14.447545


In [13]:
def mergeFeature(source): 
    d = source
    d = pd.merge(d, items_df, on=['item_id'], how='left')
    d = pd.merge(d, icats_df, on=['item_category_id'], how='left')
    d = pd.merge(d, shops_df, on=['shop_id'], how='left')

    d = pd.merge(d, train_price, on=['date_block_num','shop_id','item_id'], how='left')
    d = pd.merge(d, train_shop, on=['shop_id','item_id'], how='left')
    d = pd.merge(d, train_shop_cat, on=['shop_id','item_category_id'], how='left')
    d = pd.merge(d, train_city_cat, on=['city_id','item_category_id'], how='left')
    d = pd.merge(d, train_group, on=['item_category_group'], how='left')
    #d = pd.merge(d, train_last, on=['shop_id','item_id'], how='left')
    d = pd.merge(d, train_prev, on=['date_block_num','shop_id','item_id'], how='left')
    d = pd.merge(d, train_evm_prev, on=['date_block_num','shop_id','item_id'], how='left')
    d = pd.merge(d, train_cat_prev, on=['date_block_num','shop_id','item_category_id'], how='left')

    d.drop(['date_block_num', 'shop_id','shop_name','item_id','item_name','item_category_id','item_category_name', 'item_category_group', 'city_id'], axis=1, inplace=True)
    d.fillna(0.0, inplace=True)
    
    return d

In [14]:
def num_rescale(train, test):
    
    d = pd.concat([train, test], ignore_index=True)
    
    d['item_price_inv'] = d['item_price'].values.min()/d['item_price']
    d.drop(['item_price'], axis=1, inplace=True)
    d['item_price_inv'].fillna(0, inplace=True)
    
    columns = ['cnt_mean_shop', 'cnt_std_shop', 'cnt_mean_cat_shop', 'cnt_mean_cat_city', 'cnt_mean_group',
               'cnt_median_group', 'cnt_std_group', 'cnt_sum_prev', 'cnt_evm_s_prev', 'cnt_evm_l_prev',
               'cnt_macd_prev', 'cnt_sig_prev', 'cnt_sum_cat_prev']
    
    for column in columns:
    
        d[column] = d[column]/d[column].values.std()
    
    return d.iloc[:len(train)], d.iloc[len(train):]

In [28]:
train_set = mergeFeature(train_monthly)

X_train = train_set.drop(['item_cnt'], axis=1)
Y_train = train_set[['item_cnt']]

X_test = mergeFeature(test_df.drop(['ID'], axis=1))

X_train, X_test = num_rescale(X_train, X_test)
X_test.reset_index(drop=True, inplace=True)

X_train.to_csv("X_train.csv.gz", index=False, compression='gzip')
X_test['ID'] = test_df['ID']
X_test.to_csv("X_test.csv.gz", index=False, compression='gzip')
Y_train.to_csv("Y_train.csv.gz", index=False, compression='gzip')

After the X_train, Y_train, X_test are produced, we can work from this step